In [ ]:
#default_exp gui

# Displaying member similarity
> GUI to explore most similar members

In [ ]:
%load_ext autoreload
%autoreload 2

## Setup

In [ ]:
#export
import plotly.express as px
import plotly.graph_objects as go
import pickle
import ipywidgets as widgets
import pandas as pd
import typing, os

## Display most similar members with at least $x$ common issues voted on

Loading required files

In [ ]:
#export
path = 'similarities.pkl' if 'similarities.pkl' in os.listdir('.') else '../similarities.pkl'
with open(path, 'rb') as f:
    agreements = pickle.load(f)

In [ ]:
#export
path = 'votes.parquet' if 'votes.parquet' in os.listdir('.') else '../votes.parquet'
df = pd.read_parquet(path)

Defining widgets

In [ ]:
#export
class GUI:
    
    def __init__(self, df:pd.DataFrame, agreements:typing.Dict):
        self.df = df
        self.agreements = agreements
        self.members = tuple(df['Bezeichnung'].unique())
        self.init_widgets()
        
        
    def init_widgets(self):
        self.num_issues_widget = widgets.IntText(
            value=7,
            description='# issues',
            disabled=False
        )

        self.top_similar_widget = widgets.IntText(
            value=5,
            description='Top n',
            disabled=False
        )

        self.member_widget = widgets.Combobox(
            # value='John',
            placeholder='Choose Someone',
            options=self.members,
            description='MdB:',
            ensure_option=True,
            disabled=False
        )

        self.config_widget = widgets.Label()
        self.submit_widget = widgets.Button(description='Submit')
        self.display_widget = widgets.Output()
        
        self.submit_widget.on_click(self.on_click)
        
        
    def on_click(self, change):
        
        member = self.member_widget.value
        top_n = self.top_similar_widget.value
        min_votes = self.num_issues_widget.value
        
        self.display_widget.clear_output()
        self.config_widget.value = f'You\'ve selected {member} and are searching for, matches with {min_votes}+ common issues, limited to the top {top_n} matches.'

        # filtering down to similar members
        filtered_members = self.agreements['total_shared_votes'].loc[member]
        filtered_members = filtered_members[filtered_members >= min_votes]

        res = {outcome: self.agreements[outcome].loc[member]
               for outcome in self.agreements \
               if (outcome != 'total_shared_votes') and member in self.agreements[outcome].index}
        res = {o: (tbl[tbl.index.isin(filtered_members.index)]
                   .sort_values(ascending=False)
                   .iloc[1:])
               for o, tbl in res.items()}

        x = 'Agreement (%)'
        y = 'MdB'
        tmp_plot = pd.DataFrame(columns=[x,y,'outcome'])
        for o, tbl in res.items():
            tmp = (res[o].copy()
                   .to_frame()
                   .reset_index()
                   .rename(columns={'index':y, member:x}))
            tmp['outcome'] = o
            tmp_plot = pd.concat((tmp_plot, tmp.head(top_n).sort_values(x)),ignore_index=True)

        with self.display_widget:

            fig = px.bar(tmp_plot, x=x, y=y, facet_col='outcome',
                         title=f'Ranking by agreement (top {top_n} per outcome)')
            fig.show()
            
    def run(self):
        return widgets.VBox([widgets.Label('Select your member (MdB), the smallest number of issues they and others should have voted on and the top n similar MdBs.'),
                      self.member_widget, self.num_issues_widget, self.top_similar_widget, self.submit_widget, 
                      self.config_widget, self.display_widget])

In [ ]:
gui = GUI(df, agreements)

In [ ]:
gui.run()